# Leonardo Serilli
## mat 274426
### ML-homework3
#### 16/12/2020

In [161]:
import numpy
import matplotlib.pyplot as plt
import pandas
import math
import copy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

In [336]:
# finestra temporale per la predizione
WINDOW_SIZE = 4

# airline-passengers.csv contiene il numero di passeggeri dell'international airline
 # in unita di 1,000. I dati vanno dal gennaio del 1949 al dicembre del 1960.
df = pandas.read_csv('airline-passengers.csv', usecols=[1])
ds = df.values
# normalizzo il dataset, LSTM ha un rendimento migliore con dati normalizzati
scaler = MinMaxScaler(feature_range=(0, 1))
ds = scaler.fit_transform(ds)

In [323]:
# la funzione prende il dataset di look_back colonne (cioè il numero di passegeri dei look_back mesi precedenti al 
 # mese t) e aggiunge una colonna di output(contenente il numero di passeggeri al mese t+1)
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

In [324]:
# Calcolo lo scarto quadratico medio sul test set
def score_prediction(predictions, tst_y):
    not_normalized_predictions = scaler.inverse_transform(predictions)
    not_normalized_tst_y = scaler.inverse_transform([tst_y])
    score = math.sqrt(mean_squared_error(not_normalized_tst_y[0], not_normalized_predictions[:,0]))
    return score

In [325]:
# la funzione ritorna un dataset contenente il numero di passeggeri dal mese t-look_back al mese t
def create_input(dataset, look_back=1):
    dataX = []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
    return numpy.reshape(numpy.array(dataX), (numpy.array(dataX).shape[0], 1, numpy.array(dataX).shape[1]))

In [326]:
# creo TS e testSet
TS_size = int(len(ds) * 0.8)
tst_size = len(ds) - TS_size
TS, tst = ds[0:TS_size,:], ds[TS_size:len(ds),:]

# reshape di Train e test Set con in input il numero di passeggeri dei look_back mesi fino al mese t,
 # e in output il numero di passeggeri al mese t+1, dove t è un dato mese e t+1 il mese successivo
look_back = WINDOW_SIZE
TS_x, TS_y = create_dataset(TS, look_back)
tst_x, tst_y = create_dataset(tst, look_back)

In [327]:
# reshape dell'input per LSMT
TS_x = numpy.reshape(TS_x, (TS_x.shape[0], 1, TS_x.shape[1]))
tst_x = numpy.reshape(tst_x, (tst_x.shape[0], 1, tst_x.shape[1]))

In [256]:
#train model
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(TS_x, TS_y, epochs=100, batch_size=1)

Epoch 1/100
110/110 [==============================] - 1s 2ms/step - loss: 0.0571
Epoch 2/100
110/110 [==============================] - 0s 3ms/step - loss: 0.0166
Epoch 3/100
110/110 [==============================] - 0s 3ms/step - loss: 0.0125
Epoch 4/100
110/110 [==============================] - 0s 3ms/step - loss: 0.0097
Epoch 5/100
110/110 [==============================] - 0s 3ms/step - loss: 0.0080
Epoch 6/100
110/110 [==============================] - 0s 3ms/step - loss: 0.0066
Epoch 7/100
110/110 [==============================] - 0s 3ms/step - loss: 0.0072
Epoch 8/100
110/110 [==============================] - 0s 3ms/step - loss: 0.0080
Epoch 9/100
110/110 [==============================] - 0s 3ms/step - loss: 0.0057
Epoch 10/100
110/110 [==============================] - 0s 3ms/step - loss: 0.0066
Epoch 11/100
110/110 [==============================] - 0s 3ms/step - loss: 0.0068
Epoch 12/100
110/110 [==============================] - 0s 3ms/step - loss: 0.0061
Epoch 13/100


110/110 [==============================] - 0s 3ms/step - loss: 0.0029
Epoch 100/100
110/110 [==============================] - 0s 3ms/step - loss: 0.0025


In [333]:
# predizione per il test set
predictions = model.predict(tst_x)
score = score_prediction(predictions, tst_y)
print('Score: %.2f RMSE' % (score), ", con una window di dimensione ", WINDOW_SIZE)

Score: 55.55 RMSE , con una window di dimensione  4


In [335]:
# predizione per i prossimi k mesi
k = 10
for t in range(k):
    new_input = create_input(predictions, WINDOW_SIZE)
    pred = model.predict(new_input)
    predictions = numpy.append(predictions, [pred[-1]], axis=0)
    
    # inverto la normalizzazione e stampo i risultati
    print("# passeggeri al mese +", t+1, "=>", int(scaler.inverse_transform(predictions[-1].reshape(-1, 1))))

# passeggeri al mese + 1 => 319
# passeggeri al mese + 2 => 321
# passeggeri al mese + 3 => 323
# passeggeri al mese + 4 => 320
# passeggeri al mese + 5 => 321
# passeggeri al mese + 6 => 323
# passeggeri al mese + 7 => 320
# passeggeri al mese + 8 => 321
# passeggeri al mese + 9 => 322
# passeggeri al mese + 10 => 320
